In [40]:
from sklearn.model_selection import train_test_split

import pandas as pd 
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

In [11]:
df = pd.read_csv('../../input/clean_data/renner_web_treino.csv')
df.head()

,cod_loja,data_semana_comercial,venda
0,407,2022-06-05,188
1,407,2021-05-23,136
2,406,2021-05-30,926
3,407,2022-04-03,152
4,407,2022-07-24,33


In [12]:
df.data_semana_comercial = pd.to_datetime(df.data_semana_comercial)

In [20]:
df['ano'] = df.data_semana_comercial.dt.year
df['mes'] = df.data_semana_comercial.dt.month
df['dia'] = df.data_semana_comercial.dt.day
df = df[['cod_loja', 'data_semana_comercial', 'ano', 'mes', 'dia', 'venda']]
df.head()

,cod_loja,data_semana_comercial,ano,mes,dia,venda
0,407,2022-06-05,2022,6,5,188
1,407,2021-05-23,2021,5,23,136
2,406,2021-05-30,2021,5,30,926
3,407,2022-04-03,2022,4,3,152
4,407,2022-07-24,2022,7,24,33


In [21]:
df['semana'] = df.ano + df.mes + df.dia

In [22]:
df.semana

0      2033
1      2049
2      2056
3      2029
4      2053
       ... 
401    2036
402    2060
403    2050
404    2032
405    2040
Name: semana, Length: 406, dtype: int32

In [25]:
del df['data_semana_comercial']

In [26]:
df.corr()

,cod_loja,ano,mes,dia,venda,semana
cod_loja,1.000000,0.475158,0.019122,0.005547,-0.296100,0.094537
ano,0.475158,1.000000,-0.150828,0.002772,0.093588,0.122204
mes,0.019122,-0.150828,1.000000,0.000891,0.070456,0.333169
dia,0.005547,0.002772,0.000891,1.000000,-0.002158,0.927393
venda,-0.296100,0.093588,0.070456,-0.002158,1.000000,0.039520
semana,0.094537,0.122204,0.333169,0.927393,0.039520,1.000000


In [35]:
x = df.iloc[:,0:-1].values
y = df.iloc[:,-1:].values

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [37]:
linear_regressor = LinearRegression()
linear_regressor.fit(x_train, y_train) 


LinearRegression()

In [45]:
def sales_pipeline_v1(dataframe):
    """
        Args:
            dataframe: Pandas dataframe com os dados de vendas
        Return:
            df_web: dataframe com os dados necessários para previsão das lojas web
            df_fisica: dataframe com os dados necessários para previsão das lojas fisicas
        OBS:
            Para previsão de séries temporais, após chamar o metodo e receber os novos dataframes, é preciso transformar
            a coluna 'data_semana_comercial' em datetime novamente e coloca-lá como index do df.
        OBS2:
            Se precisar usar os dias da semana, mes ou ano, basta usar os atributos do index após transformá-lo
    """
    df_pipe = dataframe.copy()
    print('Dataframe carregado!')

    #Dropando colunas desnecessárias
    df_pipe.drop(['item', 'semana_comercial'], axis=1, inplace=True)
    print("Colunas 'item' e 'semana_comercial' dropadas!")

    #Dropando linhas com vendas Guide Shop ('GS')
    df_pipe = df_pipe.loc[df_pipe['clima'] != 'GS']
    print('Vendas Guide Shop (GS) dropadas!')

    #Alterando vendas negativas para zero
    df_pipe.loc[df_pipe.venda < 0, 'venda'] = 0
    print('Vendas negativas alteradas para zero!')

    #Transformando variaveis categóricas em variaveis numéricas
    df_pipe['localidade'] = df_pipe['localidade'].map({ 'br':2, 'uy':1, 'ar':0 })
    df_pipe['loja_tamanho'] = df_pipe['loja_tamanho'].map({ 'PP':0, 'P':1, 'M':2, 'G':3, 'GG':4 })
    df_pipe['clima'] = df_pipe['clima'].map({ 'SPO':5, 'RIO':4, 'NOR':3, 'SUL':2 ,'URU':1 ,'ARG': 0, 'W':'W'})
    print('Variaveis categoricas transformadas em numéricas!')

    #Transformando a coluna 'data_semana_comercial' em datetime
    df_pipe.data_semana_comercial = pd.to_datetime(df_pipe.data_semana_comercial)
    print("coluna 'data_semana_comercial' transformada em datetime!")

    #Tratando dataset para lojas web
    df_web = df_pipe[df_pipe['categoria_loja'] == 'web']
    del df_web['categoria_loja']
    del df_web['clima']
    del df_web['localidade']
    del df_web['loja_tamanho']
    print('Dataset de treino para lojas web criado')
    

    #Tratando dataset para lojas web
    df_fisica = df_pipe[df_pipe['categoria_loja'] == 'fisica']
    del df_fisica['categoria_loja']
    print('Dataset de treino para lojas fisicas criado')
    
    return df_web, df_fisica

In [49]:
df_teste = pd.read_csv('../../input/renner_desafio_pucrs_teste_1.csv')

In [50]:
df_teste_web, _ = sales_pipeline_v1(df_teste)

Dataframe carregado!
Colunas 'item' e 'semana_comercial' dropadas!
Vendas Guide Shop (GS) dropadas!
Vendas negativas alteradas para zero!
Variaveis categoricas transformadas em numéricas!
coluna 'data_semana_comercial' transformada em datetime!
Dataset de treino para lojas web criado
Dataset de treino para lojas fisicas criado


In [51]:
df_teste_web.head()

,cod_loja,data_semana_comercial,venda
1,407,2022-06-05,188
62,406,2022-05-15,1269
185,407,2022-06-12,129
707,406,2022-05-08,605
766,406,2022-05-29,737


In [53]:
df_teste_web['ano'] = df_teste_web.data_semana_comercial.dt.year
df_teste_web['mes'] = df_teste_web.data_semana_comercial.dt.month
df_teste_web['dia'] = df_teste_web.data_semana_comercial.dt.day
df_teste_web = df[['cod_loja', 'data_semana_comercial', 'ano', 'mes', 'dia', 'venda']]
df_teste_web['semana'] = df_teste_web.ano + df_teste_web.mes + df_teste_web.dia
df_teste_web.head()

KeyError: "['data_semana_comercial'] not in index"

In [39]:
y_pred_lr = linear_regressor.predict(x_test)

In [41]:
print(r2_score(y_test, y_pred_lr))
print(mse(y_test, y_pred_lr)**0.5)

1.0
0.0
